This notebook is created at GESIS for general public to test GIGIDL model on gender inference from images. It is a supplement to the GESIS Gender Inference website. You can upload your own images. All data is deleted afterwards.

1. Run all cells by clicking "Cell" -> "Run all" <br/>
2. Upload several photos from your computer by clicking "Upload" <br/>
3. Press "Submit" to run the model on your data. The outcome will be displayed in the table with predicted gender and confidence. The results are also saved to the csv file <br/>
4. Button "Clear" ensures no data is stored afterwards

In [ ]:
# Importing necessary libraries
from ipywidgets import VBox, HBox
import ipywidgets as widgets
from IPython.display import clear_output
from PIL import Image
import sys
from io import BytesIO
import json
import os
from IPython.display import HTML
import keras
import pandas as pd
import cv2 # importing open CV library
import face_recognition # importing face recognition library
import imutils # Package of convenience functions
import time
import dlib
import numpy as np
from pathlib import Path
import shutil 
import csv

# Reading the trained model from the folder or server
json_file = open('../GIGIDL/Trained Model/model.json', 'r')     # Loading the json file
loaded_model_json = json_file.read()
json_file.close()
loaded_model = keras.models.model_from_json(loaded_model_json)
loaded_model.load_weights("../GIGIDL/Trained Model/model.h5")    # Loading the weights

In [ ]:
HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
The raw code for this IPython notebook is by default hidden for easier reading.
To toggle on/off the raw code, click <a href="javascript:code_toggle()">here</a>.''')

In [ ]:
def reshape(numpy_array):            # Reshape the image
    d = np.stack(numpy_array)
    reshaped = d.reshape(1, d.size)
    return reshaped

def detect_faces(image):                    # Create a face detector
    face_detector = dlib.get_frontal_face_detector()
    detected_faces = face_detector(image, 1)    # Run detector and get bounding boxes of the faces on image
    face_frames = [(x.left(), x.top(),
                    x.right(), x.bottom()) for x in detected_faces]
    return face_frames

# Running the GIGIDL inference and getting the gender
def GIGIDL_inference(filename):
    try:
        a = cv2.imread(f"images/{filename}")
        detected_faces = detect_faces(a)
        for n, face_rect in enumerate(detected_faces):
            fa = Image.fromarray(a).crop(face_rect)   # Cropping the face or faces
            cropped = np.array(fa.resize((100, 100)))
            converted = cv2.cvtColor(cropped, cv2.COLOR_BGR2GRAY)    # Converting the image
            re_data = reshape(converted)
        if len(detected_faces) == 0:
            with open("output.csv", 'a+', newline='') as output: # output file is created
                wr = csv.writer(output)
                wr.writerow([filename, None, None])
                return False

        df_x = pd.DataFrame(re_data).iloc[:,0:].values.reshape(len(re_data),100,100,1)   # Creating the requird formate
        f_x = np.array(df_x)  # Need to convert as a array
            # Predicting gender and reporting the precision, recall F1-score and accuracy
        y_pred = loaded_model.predict_classes(f_x)    # Predicting the gender
        prob = np.asmatrix(loaded_model.predict(f_x))   # Calculating the probability
        g_label = ["Male", "Female"]
        def result(d):           # Finding the requird results
            if d[0,0] > d[0,1]:
                return d[0,0], g_label[0]
            if d[0,0] < d[0,1]:
                return d[0,1], g_label[1]
        gh = result(prob)
        with open("output.csv", 'a+', newline='') as output: # output file is created
            wr = csv.writer(output)
            wr.writerow([filename, gh[1], gh[0]])
    except:
        print("Oops!",sys.exc_info()[0],"occured.")


Upload images from your computer. 

In [ ]:
'''Set up layout for upload widgets'''
def initialize_widgets():
    global upload, layout
    upload = widgets.FileUpload(
        accept='image/*', 
        multiple=True  
    )
    layout = VBox([upload, submit])
    display(layout)
    
global clear, submit, submitted
submitted = False
submit = widgets.Button(description="Submit")
clear = widgets.Button(description="Clear")
initialize_widgets()

''' Clear the output and delete the files'''
def on_clear_clicked(b):
    global submitted
    clear_output()
    initialize_widgets()
    try:
        shutil.rmtree(sys.path[0]+'/images/')
    except:
        pass
    for filename in os.listdir(sys.path[0]+'/'):
        if filename.endswith(".csv"):
            os.remove(filename)
    submitted = False

'''Resize and convert images for M3. Save the file (is deleted afterwards)'''    
def preprocess_and_save(content, filename):
    stream = BytesIO(content)
    image = Image.open(stream).convert("RGB")
    stream.close()
    image = image.resize((224, 224))
    image.save(f"images/{filename}")

'''Submitting an image either as fileupload or from url'''
def on_submit_clicked(b):
    global submitted, content
    if not upload.value: #no input is given
        print('No image has been uploaded')
        return False
    Path("images/").mkdir(parents=True, exist_ok=True)
    with open('output.csv', 'w', newline='') as output:  # output file is created
        wr = csv.writer(output,quoting=csv.QUOTE_ALL)
        wr.writerow(['Filename','M3_Gender','Confidence'])
    for filename in upload.value.keys():
        content = upload.value[filename]['content'] #extracting the bytes for images
        preprocess_and_save(content, filename)
        GIGIDL_inference(filename)
    if submitted: #cleaning the previous results
        clear_output()
        initialize_widgets()
    submitted = True
    output_df = pd.read_csv("output.csv")
    print("The results are presented in the table below. NaN values indicate that no faces have been detected on a picture.")
    display(output_df, clear)

submit.on_click(on_submit_clicked)
clear.on_click(on_clear_clicked)


In [ ]:
text = HTML('<p>The results are saved in the <a href="output.csv" target="_blank">csv file</a> and the folder with photos is <a href="images" target="_blank">here</a> (in case you did not press on "Clear" yet, which deletes the stored data). </p>')
display(text)